## Parte 1

tenemos una serie de registros donde resorte que funciona y # significa que hay un resorte roto, ? significa que se desconoce.

En cada registro hay una comprobación en forma de números separados por comas donde se indica el número de grupos de resortes rotos.

se pide encontrar en número de distintas combinaciones de ? para que los números sean correctos

In [22]:
import itertools
import regex as re
from functools import lru_cache

In [15]:
def get_input(filename):
    cond_records = []
    
    with open(filename) as file:
        for line in file:
            cond_records.append(line.strip())
    
    draws = []
    counts = []
    qmarks = []
    spring_marks = []
    springs_lacking = []
    
    for record in cond_records:
        parts = record.split(' ')
        draw = parts[0]
        qmarks.append(draw.count('?'))
        spring_marks.append(draw.count('#'))
        draws.append(draw)
        
        count = [int(p) for p in parts[1].split(',')]
        counts.append(count)
        springs_lacking.append(sum(count) - draw.count('#'))
    return draws, counts, qmarks, spring_marks, springs_lacking

In [16]:
def check_line(line, count):
    groups = re.findall('#+', line)
    group_counts = [len(g) for g in groups]
    
    return group_counts == count

In [17]:
def fix_records(springs_lacking, qmarks, draws, counts):
    nlines = len(springs_lacking)
    
    count_ok = 0
    
    for i in range(nlines):
        for p in list(itertools.product(['#', '.'], repeat=qmarks[i])):
            if p.count('#') != springs_lacking[i]:
                continue
            curr_draw = draws[i]
            
            for symbol in p:
                curr_draw = curr_draw.replace('?', symbol, 1)
            
            if check_line(curr_draw, counts[i]):
                count_ok += 1
                
    return count_ok

In [19]:
draws, counts, qmarks, spring_marks, springs_lacking = get_input('test.txt')
fix_records(springs_lacking, qmarks, draws, counts)

21

In [20]:
draws, counts, qmarks, spring_marks, springs_lacking = get_input('input.txt')
fix_records(springs_lacking, qmarks, draws, counts)

7460

## Parte 2

los registros estaban doblados, hay que usar 5 copias de cada registro separadas por una interrogación "?", también hay que reemplazar las listas de resortes dañados por cinco copias separadas por una coma 

In [21]:
def get_input2(filename):
    draws = []
    counts = []
    
    with open(filename) as file:
        for line in file:
            parts = line.split(' ')
            draw = parts[0]
            #hay que usar tuplas para poder usar caches
            count = tuple(int(p) for p in parts[1].split(','))
            draws.append(draw)
            counts.append(count)
    return draws, counts

In [23]:
@lru_cache(maxsize=None)
def count_posibles(line, counts, cont_grupo):
    if len(line) == 0:
        #es una solución?
        if len(counts) == 0 and cont_grupo == 0:
            return 1
        else:
            return 0
        
    posibles = 0
    
    if line[0] == '?':
        probar = ['#', '.']
    else:
        probar = line[0]
        
    for p in probar:
        if p == '#':
            posibles += count_posibles(line[1:], counts, cont_grupo + 1)
        else:
            if cont_grupo > 0:                
                #si el grupo está completo pasa al siguiente
                if len(counts) > 0 and counts[0] == cont_grupo:                    
                    posibles += count_posibles(line[1:], counts[1:], 0)
            else:
                #si el grupo no está completo mueve hacia adelante
                posibles += count_posibles(line[1:], counts, 0)
    return posibles                    

In [24]:
def process_all(draws, counts):
    posibles = 0
    for d, c in zip(draws, counts):
        extended_d = '?'.join([d]*5)
        extended_c = c*5
        posibles += count_posibles(extended_d + '.', extended_c, 0)
    return posibles

In [25]:
draws, counts = get_input2('test.txt')
posibles = process_all(draws, counts)
posibles

525152

In [26]:
draws, counts = get_input2('input.txt')
posibles = process_all(draws, counts)
posibles

6720660274964